# 指定されたBeaconBlockのリワードを計算する

Beacon chainのAPIは以下を参照。REST APIで提供されている。
https://ethereum.github.io/beacon-APIs/

In [95]:
!pip install attrdict

In [2]:
import requests
import json
import numpy as np
from attrdict import AttrDict 

class APIClient():
    def __init__(self, host="http://192.168.100.4:3500"):
        self.host = host

    def get(self, _api):
        endpoint = "{}/{}".format(self.host, _api)
        r = requests.get(endpoint)
        res_json = r.json()
        return AttrDict(res_json)
    
    def get_block(self, slot=1):
        return self.get(f'/eth/v2/beacon/blocks/{slot}')
    
    def get_validators(self, slot=1):
        # /eth/v1/beacon/states/{state_id}/validators
        res = client.get(f"eth/v1/beacon/states/{slot}/validators")
        if hasattr(res, "data"):
            return res.data
        else:
            raise ValueError(res)

In [3]:
class BeaconStateEnum:
    def __init__(self, slot=1, host="http://192.168.100.4:3500"):
        self.client = APIClient(host=host)
        self.slot = slot
        
        self.validators = self.client.get_validators(self.slot)

    def total_active_balance(self):
        validator_statuses = set()
        
        effective_max = 32 * 10 ** 9
        total_balance = 0
        for val in self.validators:
            validator_statuses.add(val.status)
            if val.status == 'active_ongoing':
                total_balance += int(val.validator.effective_balance)

        print(f'validator_status: {validator_statuses}')
        return total_balance


In [4]:
client = APIClient()
print(client.get("eth/v1/beacon/genesis"))
slot_id=4985246

state = BeaconStateEnum(slot = slot_id)
total_balance = state.total_active_balance()
print(f"total_balance: {total_balance}")

gwei = 10 ** 9
effective_balance = 32 * gwei 

sqrt_total_balance = np.sqrt(total_balance)
base_fee_gwei = effective_balance * 64 / sqrt_total_balance / 4
print(f'gwei base base_fee: {base_fee_gwei}')

paticipate_rate = 0.928
vote_reward = base_fee_gwei * 3 * paticipate_rate
inclusion_delay = 1
inclusion_reward = base_fee_gwei * 7 / 8 * (1 / inclusion_delay)
print(f'atte reward: {vote_reward} + {inclusion_reward} = {vote_reward + inclusion_reward}')

AttrDict({'data': {'genesis_time': '1606824023', 'genesis_validators_root': '0x4b363db94e286120d76eb905340fdd4e54bfe9f06bf33ff6cf5ad27f511bfe95', 'genesis_fork_version': '0x00000000'}})
validator_status: {'withdrawal_possible', 'exited_unslashed', 'active_ongoing', 'exited_slashed', 'pending_initialized'}
total_balance: 14384226000000000
gwei base base_fee: 4269.00547721009
atte reward: 11884.911248552891 + 3735.3797925588287 = 15620.29104111172


In [28]:
EFFECTIVE_BALANCE_INCREMENT = 1_000_000_000
BASE_REWARD_FACTOR = 64

def get_base_reward_per_increment(total_active_balance):
    return EFFECTIVE_BALANCE_INCREMENT * BASE_REWARD_FACTOR // int(np.sqrt(total_active_balance))

def get_base_reward(state, validator_index):
    """
    Return the base reward for the validator defined by ``index`` with respect to the current ``state``.
    """
    increments = int(state.validators[validator_index].validator.effective_balance) // EFFECTIVE_BALANCE_INCREMENT
    return increments * get_base_reward_per_increment(state.total_active_balance())

validator = state.validators[20000]
print(validator)
base_reward = get_base_reward(state, 20000)
HEAD_WEIGHT = 14
SOURCE_WEIGHT = 14
TARGET_WEIGHT = 26
SYNC_WEIGHT = 2
PROPOSER_WEIGHT = 8

print(f"full_atte: {base_reward * (HEAD_WEIGHT + SOURCE_WEIGHT + TARGET_WEIGHT + SYNC_WEIGHT) / 64}")
print(f"not_sync_atte: {base_reward * (HEAD_WEIGHT + SOURCE_WEIGHT + TARGET_WEIGHT) / 64}")
print(f"not_head_atte: {base_reward * (SOURCE_WEIGHT + TARGET_WEIGHT + SYNC_WEIGHT) / 64}")
print(f"not_target_atte: {base_reward * (SOURCE_WEIGHT + SYNC_WEIGHT) / 64}")



AttrDict({'index': '20000', 'balance': '35971795061', 'status': 'active_ongoing', 'validator': {'pubkey': '0x93e2e3750efa79391e5d9314d01b6892dec2460e0c48cc66c3f355403afda461bafe28be68bbd490affc9e932add4805', 'withdrawal_credentials': '0x00a81b8c6eec31ff6341cdccc902c0e415cad8b33a4ac20279edb974ebeb735e', 'effective_balance': '32000000000', 'slashed': False, 'activation_eligibility_epoch': '0', 'activation_epoch': '0', 'exit_epoch': '18446744073709551615', 'withdrawable_epoch': '18446744073709551615'}})
validator_status: {'withdrawal_possible', 'exited_unslashed', 'active_ongoing', 'exited_slashed', 'pending_initialized'}
full_atte: 14924.0
not_sync_atte: 14391.0
not_head_atte: 11193.0
not_target_atte: 4264.0
penalty_atte: 7462.0


In [27]:

INACTIVITY_SCORE_BIAS = 2**2
INACTIVITY_PENALTY_QUOTIENT_ALTAIR = 3 * 2**24
INACTIVITY_PENALTY_QUOTIENT_BELLATRIX = 2**24

index = 20000
inactivity_score = 22

penalty_numerator = int(state.validators[index].validator.effective_balance) * inactivity_score
penalty_denominator = INACTIVITY_SCORE_BIAS * INACTIVITY_PENALTY_QUOTIENT_BELLATRIX
penaltie = penalty_numerator // penalty_denominator

penaltie

10490